# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [3]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [6]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [7]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [8]:
wml_credentials={
    "apikey": "lRSIS-1mK8-gQl4NpnlJhR0-8QX8vAzfW8J_DY3Ge3FQ",
  "iam_apikey_description": "Auto-generated for key e98e4872-efa4-4f58-8078-a0f1180f7d96",
  "iam_apikey_name": "Credenciales de servicio-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/0e728560389e4c928c1702d31f26450b::serviceid:ServiceId-cd58d38d-ca9f-41da-9668-834c563befb2",
  "instance_id": "71dcfa30-9596-4a7c-8fb8-ef9f80693225",
  "url": "https://us-south.ml.cloud.ibm.com"
}


 

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [9]:
client = WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [10]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Mauro", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "mauromartinezochoa@gmail.com",
               client.repository.ModelMetaNames.NAME: "Reviews classification"}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [15]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()

In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
pipeline = make_pipeline(vect, RF)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [17]:
published_model = client.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=X_train, 
                                                training_target=y_train)

2019-11-14 21:15:59,929 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [21]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [22]:
models_details = client.repository.list_models()

------------------------------------  ----------------------  ------------------------  -----------------
GUID                                  NAME                    CREATED                   FRAMEWORK
9092ebea-0497-4d80-8b40-5e1fd4a15c2f  Reviews classification  2019-11-15T00:16:03.393Z  scikit-learn-0.19
76e4f795-ee2d-4ccc-b0bd-b7d2d419b9b1  Reviews classification  2019-11-07T23:16:10.972Z  scikit-learn-0.19
------------------------------------  ----------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [23]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [24]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.6525
             precision    recall  f1-score   support

          0       0.62      0.80      0.70       200
          1       0.71      0.51      0.59       200

avg / total       0.67      0.65      0.65       400

